### Purpose 
The goal is to populate the start model for reporting
1. Populate dimentions tables ( dim_names[id,name,sex], dim_years[id, year], dim_locations[id,county])
2. Populate Fact table (fact_babynames[id, #count, id_dim_names, id_dim_years, id_dim_locations])
3. Execute a example query, only for test (not do on production)

In [ ]:
# Get notebook parameter from Azure pipeline
dbutils.widgets.text("_pipeline_run_id","0478ce36-b895-48a0-8a08-1b10430247ca")
dbutils.widgets.text("_processing_date","22-05-2024")
_pipeline_run_id = dbutils.widgets.get("_pipeline_run_id")
silver_processing_date = dbutils.widgets.get("_processing_date")
print (_pipeline_run_id)
print(silver_processing_date)

In [ ]:
# Configure my account key and account name so Databricks can access the Data Lake
accountName = dbutils.secrets.get("dataLakeScope","accountName")
accountKey = dbutils.secrets.get("dataLakeScope","accountKey")
sparkProperty = f'fs.azure.account.key.{accountName}.dfs.core.windows.net'
spark.conf.set(sparkProperty,accountKey)

In [ ]:
# Define the location of my files
goldDimNames = f'abfss://gold@{accountName}.dfs.core.windows.net/dim_names'
goldDimYears = f'abfss://gold@{accountName}.dfs.core.windows.net/dim_years'
goldDimLocations = f'abfss://gold@{accountName}.dfs.core.windows.net/dim_locations'
goldFactBabiesNames = f'abfss://gold@{accountName}.dfs.core.windows.net/fact_babynames'
silverSource = f'abfss://silver@{accountName}.dfs.core.windows.net/nybabynames'


# Table name
silver_table_name =  "silver.new_york_baby_names"
gold_dim_names_table_name =  "gold.reference_dim_names"
gold_dim_years_table_name =  "gold.reference_dim_years"
gold_dim_locations_table_name =  "gold.reference_dim_locations"
gold_fact_babynames_table_name =  "gold.fact_baby_names"

In [ ]:
# retrieve the data that has been added today. 
from pyspark.sql.functions import *

condition = f"to_date(_processing_date) == to_timestamp('{silver_processing_date}', 'dd-MM-yyyy')"
dataSilver = spark.read.table(silver_table_name).filter(condition)

display(dataSilver.printSchema)

In [ ]:
# create the schema and table, if required
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")

In [ ]:
#Dim_names table
from pyspark.sql.functions import sha2, concat
from pyspark.sql import functions as F
from delta.tables import *

# Create gold table structure
spark.sql(f"""
          CREATE EXTERNAL TABLE IF NOT EXISTS {gold_dim_names_table_name} (
              sid bigint GENERATED ALWAYS AS IDENTITY (START WITH 0 INCREMENT BY 1),
              first_name string,
              sex string, 
              _keyHash string,
              _valueHash string,
              _pipeline_run_id string,
              _processing_date timestamp)
          USING delta LOCATION '{goldDimNames}' 
          """)

# Create the align to gold table structure and add metadata
dimNameDF = dataSilver \
    .withColumn("_keyHash", sha2(concat(col("first_name"), col("sex")), 256)) \
    .withColumn("_valueHash", sha2(concat(col("_pipeline_run_id"), col("_processing_date")), 256)) \
    .drop("_input_file_modification_date") \
    .drop("_input_filename") \
    .drop("year") \
    .drop("county") \
    .drop("count")

# Preprocess the source table to eliminate multiple matches
dimNameDF = dimNameDF.groupBy("_keyHash").agg(
    F.first("first_name").alias("first_name"),
    F.first("sex").alias("sex"),
    F.first("_pipeline_run_id").alias("_pipeline_run_id"),
    F.first("_processing_date").alias("_processing_date"),
    F.first("_valueHash").alias("_valueHash")
)

# check if the goldDimNames contain the delta table
if DeltaTable.isDeltaTable(spark, goldDimNames):
    deltaTable = DeltaTable.forPath(spark, goldDimNames)
    deltaTable.alias("target").merge(
        source=dimNameDF.alias("src"),
        condition="src._keyHash = target._keyHash"
    ) \
    .whenMatchedUpdate(
        condition="src._valueHash != target._valueHash",
        set={
            "_pipeline_run_id": "src._pipeline_run_id",
            "_processing_date": "src._processing_date",
            "_keyHash": "src._keyHash",
            "_valueHash": "src._valueHash"
        }) \
    .whenNotMatchedInsert(
        values={
            "first_name": "src.first_name",
            "sex": "src.sex",
            "_pipeline_run_id": "src._pipeline_run_id",
            "_processing_date": "src._processing_date",
            "_keyHash": "src._keyHash",
            "_valueHash": "src._valueHash"
        }) \
    .execute()
else:
    # We do not want to automatically create the table from data frame because we want the identity column
    raise Exception(f"Delta table: {gold_dim_names_table_name} not found!")

In [ ]:
#Dim_years table
from pyspark.sql.types import *
from pyspark.sql import functions as F
from delta.tables import *

# Create gold table structure
spark.sql(f"""
          CREATE EXTERNAL TABLE IF NOT EXISTS {gold_dim_years_table_name} (
              sid bigint GENERATED ALWAYS AS IDENTITY (START WITH 0 INCREMENT BY 1),
              year int,
              _keyHash string,
              _valueHash string,
              _pipeline_run_id string,
              _processing_date timestamp)
          USING delta LOCATION '{goldDimYears}' 
          """)

# Create the align to gold table structure and add metadata

dimYearDF = dataSilver \
    .withColumn("_keyHash", sha2(concat(col("year")), 256)) \
    .withColumn("_valueHash", sha2(concat(col("_pipeline_run_id"), col("_processing_date")), 256)) \
    .drop("_input_file_modification_date") \
    .drop("_input_filename") \
    .drop("first_name") \
    .drop("sex") \
    .drop("county") \
    .drop("count")

# Preprocess the source table to eliminate multiple matches
dimYearDF = dimYearDF.groupBy("_keyHash").agg(
    F.first("year").alias("year"),
    F.first("_pipeline_run_id").alias("_pipeline_run_id"),
    F.first("_processing_date").alias("_processing_date"),
    F.first("_valueHash").alias("_valueHash")
)

# check if the goldDimNames contain the delta table
if(DeltaTable.isDeltaTable(spark, goldDimNames)): 

    DeltaTable.forPath(spark, goldDimYears).alias("target").merge(
        source = dimYearDF.alias("src"),
        condition = "src._keyHash = target._keyHash"
    ) \
    .whenMatchedUpdate(
        condition = "src._valueHash != target._valueHash",
        set = {
            "_pipeline_run_id" : "src._pipeline_run_id",
            "_processing_date" : "src._processing_date",
            "_keyHash" : "src._keyHash",
            "_valueHash" : "src._valueHash"
        }) \
    .whenNotMatchedInsert(
        values = {
            "year" : "src.year",
            "_pipeline_run_id" : "src._pipeline_run_id",
            "_processing_date" : "src._processing_date",
            "_keyHash" : "src._keyHash",
            "_valueHash" : "src._valueHash"
        }) \
    .execute()
else:
    # We do not want to automatically create the table from data frame because we want the identity column
    raise Exception(f"Delta table: {gold_dim_years_table_name} not found!")



In [ ]:
#Dim_location table
from pyspark.sql.types import *
from pyspark.sql import functions as F
from delta.tables import *

# Create gold table structure
spark.sql(f"""
          CREATE EXTERNAL TABLE IF NOT EXISTS {gold_dim_locations_table_name} (
              sid bigint GENERATED ALWAYS AS IDENTITY (START WITH 0 INCREMENT BY 1),
              county string,
              _keyHash string,
              _valueHash string,
              _pipeline_run_id string,
              _processing_date timestamp)
          USING delta LOCATION '{goldDimLocations}' 
          """)

# Create the align to gold table structure and add metadata

dimLocationDF = dataSilver \
    .withColumn("_keyHash", sha2(concat(col("county")), 256)) \
    .withColumn("_valueHash", sha2(concat(col("_pipeline_run_id"), col("_processing_date")), 256)) \
    .drop("_input_file_modification_date") \
    .drop("_input_filename") \
    .drop("first_name") \
    .drop("sex") \
    .drop("year") \
    .drop("count")

# Preprocess the source table to eliminate multiple matches
dimLocationDF = dimLocationDF.groupBy("_keyHash").agg(
    F.first("county").alias("county"),
    F.first("_pipeline_run_id").alias("_pipeline_run_id"),
    F.first("_processing_date").alias("_processing_date"),
    F.first("_valueHash").alias("_valueHash")
)

# check if the goldDimNames contain the delta table
if(DeltaTable.isDeltaTable(spark, goldDimNames)): 

    DeltaTable.forPath(spark, goldDimLocations).alias("target").merge(
        source = dimLocationDF.alias("src"),
        condition = "src._keyHash = target._keyHash"
    ) \
    .whenMatchedUpdate(
        condition = "src._valueHash != target._valueHash",
        set = {
            "_pipeline_run_id" : "src._pipeline_run_id",
            "_processing_date" : "src._processing_date",
            "_keyHash" : "src._keyHash",
            "_valueHash" : "src._valueHash"
        }) \
    .whenNotMatchedInsert(
        values = {
            "county" : "src.county",
            "_pipeline_run_id" : "src._pipeline_run_id",
            "_processing_date" : "src._processing_date",
            "_keyHash" : "src._keyHash",
            "_valueHash" : "src._valueHash"
        }) \
    .execute()
else:
    # We do not want to automatically create the table from data frame because we want the identity column
    raise Exception(f"Delta table: {gold_dim_locations_table_name} not found!")



In [ ]:
# Create fact table
from pyspark.sql.types import *
from delta.tables import *

spark.sql(f"""
          CREATE EXTERNAL TABLE IF NOT EXISTS {gold_fact_babynames_table_name} (
              sid bigint GENERATED ALWAYS AS IDENTITY (START WITH 0 INCREMENT BY 1),
              nameSid bigint,
              yearSid bigint,
              locationSid string,
              count int,
              _keyHash string,
              _valueHash string,
              _pipeline_run_id string,
              _processing_date timestamp)
          USING delta LOCATION '{goldFactBabiesNames}' 
          """)

# get only the required columns from dim tables
dataDimNames = spark.read.table(gold_dim_names_table_name).select("first_name","sex", col("sid").alias("nameSid"))
dataDimYears = spark.read.table(gold_dim_years_table_name).select("year",col("sid").alias("yearSid"))
dataDimLocations = spark.read.table(gold_dim_locations_table_name).select("county", col("sid").alias("locationSid"))

# Join all the data to find the matching SID
mappedData = dataSilver \
    .join(dataDimNames, (dataSilver.first_name == dataDimNames.first_name) & (dataSilver.sex == dataDimNames.sex), how="left") \
    .join(dataDimYears, (dataSilver.year == dataDimYears.year), how="left") \
    .join(dataDimLocations, (dataSilver.county == dataDimLocations.county), how="left") 

mappedData = mappedData.select("nameSid", "yearSid", "locationSid", "count","_pipeline_run_id","_processing_date")

# if any unmatch, the value = -1.
mappedData = mappedData \
        .withColumn("nameSid", when(col("nameSid").isNull(), lit(-1)).otherwise(col("nameSid"))) \
        .withColumn("yearSid", when(col("yearSid").isNull(), lit(-1)).otherwise(col("yearSid"))) \
        .withColumn("locationSid", when(col("locationSid").isNull(), lit(-1)).otherwise(col("locationSid")))

# All the SIDs are part of the keyHash

dataFact = mappedData \
    .withColumn("_keyHash", sha2(concat(col("nameSid"), col("yearSid"), col("locationSid")), 256)) \
    .withColumn("_valueHash", sha2(concat(col("count"),col("_pipeline_run_id"),col("_processing_date")), 256))

# Preprocess the source table to eliminate multiple matches
dataFact = dataFact.groupBy("_keyHash").agg(
    F.first("nameSid").alias("nameSid"),
    F.first("yearSid").alias("yearSid"),
    F.first("locationSid").alias("locationSid"),
    F.first("count").alias("count"),
    F.first("_pipeline_run_id").alias("_pipeline_run_id"),
    F.first("_processing_date").alias("_processing_date"),
    F.first("_valueHash").alias("_valueHash")
)

# check if the goldFactBabiesNames contain the delta table
if(DeltaTable.isDeltaTable(spark, goldFactBabiesNames)): 

    DeltaTable.forPath(spark, goldFactBabiesNames).alias("target").merge(
        source = dataFact.alias("src"),
        condition = "src._keyHash = target._keyHash"
    ) \
    .whenMatchedUpdate(
        condition = "src._valueHash != target._valueHash",
        set = {
            "count" : "src.count",
            "_pipeline_run_id" : "src._pipeline_run_id",
            "_processing_date" : "src._processing_date",
            "_valueHash" : "src._valueHash"
        }) \
    .whenNotMatchedInsert(
        values = {
            "nameSid" : "src.nameSid",
            "yearSid" : "src.yearSid",
            "locationSid" : "src.locationSid",
            "count" : "src.count",
            "_pipeline_run_id" : "src._pipeline_run_id",
            "_processing_date" : "src._processing_date",
            "_keyHash" : "src._keyHash",
            "_valueHash" : "src._valueHash"
        }) \
    .execute()
else:
    # We do not want to automatically create the table from data frame because we want the identity column
    raise Exception(f"Delta table: {gold_fact_babynames_table_name} not found!")

In [ ]:
%sql
-- three most popular names in 2021
-- Check your result for testing. Do not do this in production!

--SELECT n.first_name, SUM(f.count) AS total_count
--FROM gold.fact_baby_names f
--JOIN gold.reference_dim_names n ON f.nameSid = n.sid
--JOIN gold.reference_dim_years y ON f.yearSid = y.sid
--WHERE n.sex = 'M' AND y.year = 2021
--GROUP BY n.first_name
---ORDER BY total_count DESC
--LIMIT 3;



first_name,total_count
LIAM,404
NOAH,246
LUCAS,136


Databricks visualization. Run in Databricks to view.

In [ ]:
#  Maintenance for Data Table

# To optimized the performance of the Delta Table, we need to execute 2 commands:
# 1. optimize(): Optimize the number of files used to store the data.
# 2. vacuum(): remove the ild version of the data. It reduce the overhead but it limites the version we can go back to.


# Databricks recommends frequently running the OPTIMIZE command to compact small files.
# This operation does not remove the old files. To remove them, run the VACUUM command (https://learn.microsoft.com/en-us/azure/databricks/delta/vacuum).
# https://learn.microsoft.com/en-us/azure/databricks/delta/best-practices#--compact-files

# In azure we could do predictive optimization (https://learn.microsoft.com/en-us/azure/databricks/optimizations/predictive-optimization#what-operations-does-predictive-optimization-run), it have prerequisites, like a premium plan and managed tables(https://learn.microsoft.com/en-us/azure/databricks/optimizations/predictive-optimization#prerequisites-for-predictive-optimization)

gridDataDelta = DeltaTable.forName(spark, gold_dim_names_table_name)

# In this example, we will run and vacuum every 30 days
if gridDataDelta.history(30).filter("operation = 'VACUUM START'").count() == 0:
      gridDataDelta.optimize()
      gridDataDelta.vacuum() # default = 7 days

gridDataDelta = DeltaTable.forName(spark, gold_dim_years_table_name)

# In this example, we will run and vacuum every 30 days
if gridDataDelta.history(30).filter("operation = 'VACUUM START'").count() == 0:
      gridDataDelta.optimize()
      gridDataDelta.vacuum() # default = 7 days

gridDataDelta = DeltaTable.forName(spark, gold_dim_locations_table_name)

# In this example, we will run and vacuum every 30 days
if gridDataDelta.history(30).filter("operation = 'VACUUM START'").count() == 0:
      gridDataDelta.optimize()
      gridDataDelta.vacuum() # default = 7 days

gridDataDelta = DeltaTable.forName(spark, gold_fact_babynames_table_name)

# In this example, we will run and vacuum every 30 days
if gridDataDelta.history(30).filter("operation = 'VACUUM START'").count() == 0:
      gridDataDelta.optimize()
      gridDataDelta.vacuum() # default = 7 days
